In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gym
import base
from src.agents import DeepQAgent
from src.util import JupyterCallback

# Environment

In [ ]:
env = gym.make('Breakout-v0')

In [ ]:
env.observation_space

In [ ]:
env.action_space

# Agent

In [ ]:
agent = DeepQAgent(env, frames_per_action=4, replay_size=250000)
agent

In [ ]:
res_random = agent.play()
res_random

In [ ]:
res_random.mean()

In [ ]:
agent.observe(50000)

In [ ]:
agent.train(episodes=15000, callback=JupyterCallback())

In [ ]:
res_trained = agent.play()
res_trained

In [ ]:
res_trained.mean()

In [ ]:
agent.save_model()